In [1]:
import zipfile
import pandas as pd

# California

In [2]:
df_ca = pd.read_pickle("../data/ca_cleaned.pkl", compression= "gzip")

In [3]:
df_ca.head()

,date,date_dt,county_name,subject_race,subject_sex,search_conducted,search_basis,outcome
0,2009-07-01,2009-07-01,Stanislaus County,other,male,False,no_search,unknown
1,2009-07-01,2009-07-01,Stanislaus County,hispanic,female,False,no_search,summons
2,2009-07-01,2009-07-01,Stanislaus County,hispanic,female,True,other,summons
3,2009-07-01,2009-07-01,Stanislaus County,white,female,False,no_search,summons
4,2009-07-01,2009-07-01,Stanislaus County,hispanic,male,True,other,summons


In [4]:
df_ca["date"] = pd.to_datetime(df_ca["date"])

In [5]:
df_ca["year"] = df_ca["date"].dt.year

There are some spelling mistakes in county names and suffixe County to delete at the end of the string.

In [6]:
df_ca['county_name'] = df_ca['county_name'].str[:-7]
df_ca['county_name'] = df_ca['county_name'].apply(lambda x : x.replace("Siskyou", "Siskiyou"))
df_ca['county_name'] = df_ca['county_name'].apply(lambda x : x.replace("Santa Barbera", "Santa Barbara"))

## Ethnicity

In [7]:
count_year_ethnicity_ca = df_ca.groupby(["year", "county_name", "subject_race"]).count()["date"]
count_year_ethnicity_ca = pd.DataFrame(count_year_ethnicity_ca)
count_year_ethnicity_ca = count_year_ethnicity_ca.rename(columns = {"date": "nb_arrest"})

count_year_ethnicity_ca.head(10)


nb_arrest
year county_name subject_race                     
2009 Alameda     asian/pacific islander      13886
                 black                       20976
                 hispanic                    25689
                 other                       13071
                 white                       40816
     Alpine      asian/pacific islander        341
                 black                         125
                 hispanic                      661
                 other                         348
                 white                        5630

In [8]:
count_year_ethnicity_ca = count_year_ethnicity_ca.reset_index()

## Gender

In [9]:
count_year_gender_ca = df_ca.groupby(["year", "county_name", "subject_sex"]).count()["date"]
count_year_gender_ca = pd.DataFrame(count_year_gender_ca)
count_year_gender_ca = count_year_gender_ca.rename(columns = {"date": "nb_arrest"})

count_year_gender_ca.head(10)

nb_arrest
year county_name subject_sex           
2009 Alameda     female           34786
                 male             79651
     Alpine      female            1895
                 male              5210
     Amador      female            2239
                 male              4773
     Butte       female            6795
                 male             11053
     Calaveras   female            3282
                 male              6266

In [10]:
count_year_gender_ca = count_year_gender_ca.reset_index()

In [11]:
count_year_ethnicity_ca.to_pickle("../data/count_year_ethnicity_ca.pkl", compression="gzip")
count_year_gender_ca.to_pickle("../data/count_year_gender_ca.pkl", compression="gzip")


# Population

In [12]:
population_ca = pd.read_csv('../data/population_ca.csv')
population_ca.head()

,County,City,Incorportation_date,pop_april_1980,pop_april_1990,pop_april_2000,pop_april_2010
0,San Bernardino,Adelanto,1970,2164.0,8517,18130,31765
1,Los Angeles,Agoura Hills,1982,NaN,20390,20537,20330
2,Alameda,Alameda,1854,63852.0,76459,72259,73812
3,Alameda,Albany,1908,15130.0,16327,16444,18539
4,Los Angeles,Alhambra,1903,64767.0,82106,85804,83089


In [13]:
population_ca = population_ca.groupby('County')['pop_april_2010'].agg(['sum'])
population_ca.rename(columns={'sum' : 'population'},inplace = True)

#### Ethni

In [14]:
relative_count_ethnicity_ca = pd.merge(count_year_ethnicity_ca, population_ca, left_on = 'county_name', right_on = 'County')
relative_count_ethnicity_ca['relative_arrest'] = relative_count_ethnicity_ca['nb_arrest'] / relative_count_ethnicity_ca['population']

#### Gender

In [17]:
relative_count_gender_ca = pd.merge(count_year_gender_ca, population_ca, left_on = 'county_name', right_on = 'County')
relative_count_gender_ca['relative_arrest'] = relative_count_gender_ca['nb_arrest'] / relative_count_gender_ca['population']

-----

# Texas

In [20]:
df_tx = pd.read_pickle("../data/tx_cleaned.pkl", compression= "gzip")
df_tx.head()

,date,date_dt,county_name,subject_race,subject_sex,search_conducted,search_basis,outcome
7647464,2009-01-01,2009-01-01,Bee County,white,female,False,no_search,warning
7647465,2009-01-01,2009-01-01,Harris County,white,male,False,no_search,warning
7647466,2009-01-01,2009-01-01,Zapata County,hispanic,male,False,no_search,citation
7647467,2009-01-01,2009-01-01,Val Verde County,hispanic,male,False,no_search,warning
7647468,2009-01-01,2009-01-01,Howard County,white,female,False,no_search,warning


In [21]:
df_tx["date"] = pd.to_datetime(df_tx["date"])
df_tx["year"] = df_tx["date"].dt.year
df_tx['county_name'] = df_tx['county_name'].str[:-7]

## Ethnicity

In [22]:
count_year_ethnicity_tx = df_tx.groupby(["year", "county_name", "subject_race"]).count()["date"]
count_year_ethnicity_tx = pd.DataFrame(count_year_ethnicity_tx)
count_year_ethnicity_tx = count_year_ethnicity_tx.rename(columns = {"date": "nb_arrest"})

count_year_ethnicity_tx.head(10)


nb_arrest
year county_name subject_race                     
2009 Anderson    asian/pacific islander         57
                 black                        1557
                 hispanic                     1029
                 other                          13
                 unknown                       148
                 white                        7235
     Andrews     asian/pacific islander         17
                 black                          97
                 hispanic                     1612
                 other                           2

In [23]:
count_year_ethnicity_tx = count_year_ethnicity_tx.reset_index()

## Gender

In [24]:
count_year_gender_tx = df_tx.groupby(["year", "county_name", "subject_sex"]).count()["date"]
count_year_gender_tx = pd.DataFrame(count_year_gender_tx)
count_year_gender_tx = count_year_gender_tx.rename(columns = {"date": "nb_arrest"})

count_year_gender_tx.head(10)

nb_arrest
year county_name subject_sex           
2009 Anderson    female            3618
                 male              6421
     Andrews     female             969
                 male              2663
     Angelina    female            4419
                 male              8365
     Aransas     female            1243
                 male              2882
     Archer      female            1918
                 male              3524

In [25]:
count_year_gender_tx = count_year_gender_tx.reset_index()

## Population

In [26]:
population_tx = pd.read_csv('../data/population_tx.csv', thousands=',')
population_tx.drop(['FIPS','2010_Area_sqmi','2010_PopDensity'], axis = 1, inplace = True)
population_tx.rename(columns = {'2010_CensusPopulation' : 'population'}, inplace = True)
population_tx.head()

,County,population
0,Dallas,2368139
1,Harris,4092459
2,Tarrant,1809034
3,Bexar,1714773
4,Travis,1024266


In [37]:
relative_count_ethnicity_tx = pd.merge(count_year_ethnicity_tx, population_tx, left_on = 'county_name', right_on = 'County').drop(['County'], axis = 1)
relative_count_ethnicity_tx['relative_arrest'] = relative_count_ethnicity_tx['nb_arrest'] / relative_count_ethnicity_tx['population']

In [48]:
relative_count_gender_tx = pd.merge(count_year_gender_tx, population_tx, left_on = 'county_name', right_on = 'County').drop(['County'], axis = 1)
relative_count_gender_tx['relative_arrest'] = relative_count_gender_tx['nb_arrest'] / relative_count_gender_tx['population']

# All

In [41]:
arrest_ethnicity = pd.concat([relative_count_ethnicity_ca, relative_count_ethnicity_tx])

In [44]:
arrest_ethnicity

,year,county_name,subject_race,nb_arrest,population,relative_arrest
0,2009,Alameda,asian/pacific islander,13886,1369005,0.010143
1,2009,Alameda,black,20976,1369005,0.015322
2,2009,Alameda,hispanic,25689,1369005,0.018765
3,2009,Alameda,other,13071,1369005,0.009548
4,2009,Alameda,white,40816,1369005,0.029814
...,...,...,...,...,...,...
11656,2016,Zavala,asian/pacific islander,24,11677,0.002055
11657,2016,Zavala,black,98,11677,0.008393
11658,2016,Zavala,hispanic,4845,11677,0.414918
11659,2016,Zavala,other,4,11677,0.000343


In [49]:
arrest_gender = pd.concat([relative_count_gender_ca, relative_count_gender_tx])
arrest_gender.head()

,year,county_name,subject_sex,nb_arrest,population,relative_arrest
0,2009,Alameda,female,34786,1369005,0.025410
1,2009,Alameda,male,79651,1369005,0.058182
2,2010,Alameda,female,76046,1369005,0.055548
3,2010,Alameda,male,169080,1369005,0.123506
4,2011,Alameda,female,87530,1369005,0.063937


In [50]:
arrest_ethnicity.to_csv('../data/arrest_ethnicity.csv')
arrest_gender.to_csv('../data/arrest_gender.csv')